#Loading the data and cleaning

In [ ]:
import math
from collections import Counter
import operator
import numpy as np
import re
import optparse
import os 
import glob
import sys 
#Reading the documents
files = glob.glob(os.path.join('', '*'))
#print(files)
doc_numbers = list()
text = list()
title=list()
# Cleaning the documents
for file in files:
    # Open the file
    
    try:
        with open(file, 'r', encoding='utf-8') as f:
            # Read the contents of the file
            content = f.read()
            t=content.split('\n')
            st =t[1]
            st=st.replace('      ', '')
            title.append(st)
            stripped_content = content.replace('\n', ' ') 
            stripped_content = stripped_content.replace('     ', '')
            stripped_content = stripped_content.replace('TITLE:','')
            stripped_content = stripped_content.replace('SUMMARY:','')
            stripped_content = stripped_content.replace('DETAILED DESCRIPTION:','')
            stripped_content = stripped_content.replace('ELIGIBILITY CRITERIA:','')
            stripped_content = stripped_content.replace('Inclusion Criteria:','')
            stripped_content = stripped_content.replace('Exclusion Criteria:','')
            doc_numbers.append(file)
            text.append(stripped_content)
            DOCUMENT = stripped_content
            break
            
    except:
        continue



# Text Preprocessing


In [ ]:
import re

DOCUMENT = re.sub(r'\n|\r', ' ', DOCUMENT)
DOCUMENT = re.sub(r' +', ' ', DOCUMENT)

DOCUMENT = DOCUMENT.strip()


In [ ]:
!pip install gensim==3.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from gensim.summarization import summarize
#Using the Gensim Summarizer

summarize(DOCUMENT, ratio=0.9, split=False)

'This study will develop and evaluate an approach to low back pain that allows subjects to talk with each other and with health professionals via an Internet discussion group.\nResults we will look at include health behaviors, such as exercise; health status, such as pain and disability; and health care use, such as number of visits to doctors and other health care providers.\nAnyone 18 years old or older who lives in the United States and has ongoing Internet access can take part in the study.\nAll subjects must have back pain and meet the eligibility criteria listed below.\nThis study will develop and evaluate in a randomized trial a low back pain intervention that allows subjects to talk with each other and with health professionals via an Internet discussion group.\nThe intervention consists of a book and a videotape and is based on interaction with other participants in the program and health professionals through a closed password protected moderated Internet discussion group.\nO

In [ ]:
summary = summarize(DOCUMENT, word_count=100, split=False)

summary

'This study will develop and evaluate an approach to low back pain that allows subjects to talk with each other and with health professionals via an Internet discussion group.\nResults we will look at include health behaviors, such as exercise; health status, such as pain and disability; and health care use, such as number of visits to doctors and other health care providers.\nThis study will develop and evaluate in a randomized trial a low back pain intervention that allows subjects to talk with each other and with health professionals via an Internet discussion group.'

In [ ]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge
rouge = Rouge()
reference_summary = "This study will develop and evaluate an approach to low back pain that allows subjects to talk with each other and with health professionals via an Internet discussion group. Outcome measures include health behaviors, health status, and health care use. Must have access to a computer with e-mail, be 18 years old, and have seen a doctor for back pain at least once in the past year. Back pain or sciatica can lead to severe limitations, legal proceedings, bladder/bowel control issues, and numbness in the crotch."
scores = rouge.get_scores(summary, reference_summary)
print("ROUGE-1 Precision:", scores[0]['rouge-1']['p'])
print("ROUGE-1 Recall:", scores[0]['rouge-1']['r'])
print("ROUGE-1 F1-score:", scores[0]['rouge-1']['f'])

print("ROUGE-2 Precision:", scores[0]['rouge-2']['p'])
print("ROUGE-2 Recall:", scores[0]['rouge-2']['r'])
print("ROUGE-2 F1-score:", scores[0]['rouge-2']['f'])

print("ROUGE-L Precision:", scores[0]['rouge-l']['p'])
print("ROUGE-L Recall:", scores[0]['rouge-l']['r'])
print("ROUGE-L F1-score:", scores[0]['rouge-l']['f'])


ROUGE-1 Precision: 0.9
ROUGE-1 Recall: 0.4090909090909091
ROUGE-1 F1-score: 0.5624999957031251
ROUGE-2 Precision: 0.7567567567567568
ROUGE-2 Recall: 0.3373493975903614
ROUGE-2 F1-score: 0.46666666240138893
ROUGE-L Precision: 0.8666666666666667
ROUGE-L Recall: 0.3939393939393939
ROUGE-L F1-score: 0.5416666623697917


In [ ]:
import nltk
nltk.download('punkt')
sentences = nltk.sent_tokenize(DOCUMENT)

len(sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


11

In [ ]:
import numpy as np
import re

nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    tokens = nltk.word_tokenize(doc)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
norm_sentences = normalize_corpus(sentences)
norm_sentences[:3]

array(['low back pain patient education evaluation back pain one common symptoms',
       'also great cause days lost work visits health care providers',
       'study develop evaluate approach low back pain allows subjects talk health professionals via internet discussion group'],
      dtype='<U771')

# Text Representation with Feature Engineering

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
dt_matrix = tv.fit_transform(norm_sentences)
dt_matrix = dt_matrix.toarray()

vocab = tv.get_feature_names_out()
td_matrix = dt_matrix.T
print(td_matrix.shape)
pd.DataFrame(np.round(td_matrix, 2), index=vocab).head(10)

(137, 11)


,0,1,2,3,4,5,6,7,8,9,10
access,0.00,0.00,0.00,0.0,0.26,0.00,0.00,0.0,0.0,0.0,0.12
accident,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.07
activities,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.07
age,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.07
allows,0.00,0.00,0.28,0.0,0.00,0.00,0.26,0.0,0.0,0.0,0.00
also,0.00,0.36,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00
anyone,0.00,0.00,0.00,0.0,0.31,0.00,0.00,0.0,0.0,0.0,0.00
approach,0.00,0.00,0.33,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00
area,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.07
back,0.41,0.00,0.20,0.0,0.00,0.25,0.19,0.0,0.0,0.0,0.42


# Latent Semantic Analysis

In [ ]:
from scipy.sparse.linalg import svds
    
def low_rank_svd(matrix, singular_count=2):
    u, s, vt = svds(matrix, k=singular_count)
    return u, s, vt

In [ ]:
num_sentences = 8
num_topics = 3

u, s, vt = low_rank_svd(td_matrix, singular_count=num_topics)  
print(u.shape, s.shape, vt.shape)
term_topic_mat, singular_values, topic_document_mat = u, s, vt

(137, 3) (3,) (3, 11)


In [ ]:
# remove singular values below threshold                                         
sv_threshold = 0.5
min_sigma_value = max(singular_values) * sv_threshold
singular_values[singular_values < min_sigma_value] = 0

In [ ]:
salience_scores = np.sqrt(np.dot(np.square(singular_values), 
                                 np.square(topic_document_mat)))
salience_scores

array([0.6675744 , 0.5311254 , 0.88891539, 0.93516341, 0.19559259,
       0.65048121, 0.92404369, 0.55312878, 0.93516341, 0.36613533,
       0.73790598])

In [ ]:
top_sentence_indices = (-salience_scores).argsort()[:num_sentences]
top_sentence_indices.sort()

In [ ]:
print('\n'.join(np.array(sentences)[top_sentence_indices]))

Low Back Pain Patient Education Evaluation Back pain is one of the most common of all symptoms.
This study will develop and evaluate an approach to low back pain that allows subjects to talk with each other and with health professionals via an Internet discussion group.
Results we will look at include health behaviors, such as exercise; health status, such as pain and disability; and health care use, such as number of visits to doctors and other health care providers.
All subjects must have back pain and meet the eligibility criteria listed below.
This study will develop and evaluate in a randomized trial a low back pain intervention that allows subjects to talk with each other and with health professionals via an Internet discussion group.
The intervention consists of a book and a videotape and is based on interaction with other participants in the program and health professionals through a closed password protected moderated Internet discussion group.
Outcome measures include health 

In [ ]:
similarity_matrix = np.matmul(dt_matrix, dt_matrix.T)
print(similarity_matrix.shape)
np.round(similarity_matrix, 3)

(11, 11)


array([[1.   , 0.   , 0.201, 0.044, 0.   , 0.172, 0.188, 0.   , 0.044,
        0.   , 0.282],
       [0.   , 1.   , 0.03 , 0.289, 0.   , 0.   , 0.028, 0.024, 0.289,
        0.   , 0.017],
       [0.201, 0.03 , 1.   , 0.107, 0.102, 0.16 , 0.835, 0.202, 0.107,
        0.058, 0.135],
       [0.044, 0.289, 0.107, 1.   , 0.   , 0.027, 0.1  , 0.07 , 0.869,
        0.   , 0.072],
       [0.   , 0.   , 0.102, 0.   , 1.   , 0.   , 0.096, 0.037, 0.   ,
        0.   , 0.108],
       [0.172, 0.   , 0.16 , 0.027, 0.   , 1.   , 0.15 , 0.   , 0.027,
        0.   , 0.265],
       [0.188, 0.028, 0.835, 0.1  , 0.096, 0.15 , 1.   , 0.236, 0.1  ,
        0.211, 0.126],
       [0.   , 0.024, 0.202, 0.07 , 0.037, 0.   , 0.236, 1.   , 0.07 ,
        0.107, 0.005],
       [0.044, 0.289, 0.107, 0.869, 0.   , 0.027, 0.1  , 0.07 , 1.   ,
        0.   , 0.072],
       [0.   , 0.   , 0.058, 0.   , 0.   , 0.   , 0.211, 0.107, 0.   ,
        1.   , 0.059],
       [0.282, 0.017, 0.135, 0.072, 0.108, 0.265, 0.126, 0.0